In [49]:
import networkx as nx
import math
import numpy as np

In [15]:
#读取
def get_seed():
    G1 = nx.read_edgelist('seed_G1.edgelist')
    G2 = nx.read_edgelist('seed_G2.edgelist')
    with open('seed_node_pairs.txt','r') as sp:
        lines = sp.readlines()
    return G1,G2,lines


In [16]:
#获得G1 匹配的  与 G2 匹配的 并存入字典
def get_pair(lines):
    g1p = []
    g1pdir = {}
    g2p =[]
    for line in lines:
        line = line.strip('\n')
        line = line.split(' ')
        g1p.append(line[0])
        g2p.append(line[1])
        g1pdir[line[0]] = line[1]
    return g1p,g2p,g1pdir

In [17]:

#获得未匹配过的 G1 与 G2 的 点
def get_unpair(G1,G2):
    g1np = []
    g2np = []
    for i in G1.nodes:
        if i not in g1p:
            g1np.append(i)
    for j in G2.nodes:
        if j not in g2p:
            g2np.append(j)
    return g1np,g2np

In [22]:
def get_degree(node_list,G):
    degree_dir = {}
    for i in node_list:
        degree_dir[i] = G.degree(i)
    return degree_dir

In [34]:
def check_in_pairs(node,G,pair_list):
    in_pair_list =[]
    for neighbor in G.neighbors(node):
        if neighbor in pair_list:
            in_pair_list.append(neighbor)
    return in_pair_list      

In [41]:
def check_in_other_neighbor(neighbor_pair_list,other_G,pair_dir,other_node):
    other_pair_list = []
    for i in neighbor_pair_list:
        other_pair_list.append(pair_dir[i])
    count = 0
    for j in other_pair_list:
        if j in other_G.neighbors(other_node):
            count +=1
    return count

In [47]:
def get_score(node1_degree,node2_degree,count):
    score = count/((math.sqrt(node1_degree))*(node2_degree))
    return score

In [75]:
def get_ECCE (score_list):
    std = np.std(score_list)
    max1 = max(score_list)
    score_list.remove(max1)
    max2 = max(score_list)
    reslut = (max1-max2)/std
    return reslut

In [23]:
if __name__=="__main__":
    G1,G2,node_pairs = get_seed()
    G1_pair,G2_pair,G1_pair_dir = get_pair(node_pairs)
    G1_unpair,G2_unpair = get_unpair(G1,G2)
    G1_unpair_degree_dir = get_degree(G1_unpair,G1)
    G2_unpair_degree_dir = get_degree(G2_unpair,G2)
    

In [48]:
l = check_in_pairs('500',G1,G1_pair)

count =check_in_other_neighbor(l,G2,G1_pair_dir,'1684')
score = get_score(G1_unpair_degree_dir['500'],G2_unpair_degree_dir['1684'],count)
print(score)

0.0035355339059327377


In [82]:
a = 1/(2*math.sqrt(2))
b =0
c = 1/2
d = [b,a,c]
e = np.std(d)
print(d)
print()
print(np.std(d))
print()
print(get_ECCE(d))

[0, 0.35355339059327373, 0.5]

0.20988002112496087

0.6977634584834215


In [83]:
a = 1/(2*math.sqrt(2))
b =0
c = 1/4
d = [b,a,c]
e = np.std(d)
print(d)
print()
print(np.std(d))
print()
print(get_ECCE(d))

[0, 0.35355339059327373, 0.25]

0.14840758617303568

0.6977634584834211
